<a href="https://colab.research.google.com/github/kupieckozienice/pyPDF2/blob/main/zapytanie_RFX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/kupieckozienice/pyPDF2 cloned-repo
%cd cloned-repo
!ls

In [ ]:
pip install python-docx

In [ ]:
pip install PyPDF2

In [ ]:
pip install openpyxl

In [ ]:
pip install pdfminer.six

In [ ]:
os.chdir("/content")
print(os.getcwd())

In [ ]:
import os
import pathlib as pth
from typing import Union, Literal, List
from PyPDF2 import PdfReader, PdfWriter
from datetime import datetime  # Import the datetime module
from pdfminer.high_level import extract_text as fallback_text_extraction

In [ ]:
def convertPdfDatetime(pd):
    from datetime import datetime
    dtformat = "%Y%m%d%H%M%S"
    clean = pd.replace("D:","").split('+')[0]
    return datetime.strptime(clean,dtformat)

In [ ]:
# add blank page
def add_blank_page(writer, path_destination):
  writer.add_blank_page(width=200, height=200)

  data = b"any bytes - typically read from a file"
  writer.add_attachment("smile.png", data)

  with open("output.pdf", "wb") as output_stream:
      writer.write(output_stream)

In [ ]:
def check_if_encrypted(path_to_file):
  if os.path.isfile(path_to_file):
    reader = PdfReader(path_to_file)

    if reader.is_encrypted:
      writer = PdfWriter()
      password = input("Enter your password: ")
      reader.decrypt(password)

      # Add all pages to the writer
      for page in reader.pages:
          writer.add_page(page)

      # Save the new PDF to a file
      with open(path_to_file, "wb") as f:
          writer.write(f)

      print("plik rozszyfrowany")
    else:
      print("plik NIE był zaszyfrowany")

In [ ]:
def annots_reading(reader):
  print("funkcja annots_reading załadowana")

  attachments = {}

  for page in reader.pages:
    if "/Annots" in page:
        for annot in page["/Annots"]:
            obj = annot.get_object()
            annotation = {"subtype": obj["/Subtype"], "location": obj["/Rect"]}
            print(annotation)

            # text
            subtype = annot.get_object()["/Subtype"]
            if subtype == "/Text":
                print(annot.get_object()["/Contents"])

            # Highlights
            subtype = annot.get_object()["/Subtype"]
            if subtype == "/Highlight":
                coords = annot.get_object()["/QuadPoints"]
                x1, y1, x2, y2, x3, y3, x4, y4 = coords

            # # attachments
            # subtype = annot.get_object()["/Subtype"]
            # if subtype == "/FileAttachment":
            #     fileobj = annotobj["/FS"]
            #     attachments[fileobj["/F"]] = fileobj["/EF"]["/F"].get_data()
    else:
      # print("brak andotacji w pliku")
      pass

In [ ]:
# iterate over files in directory

def iterate_over_files_in_directory(path_to_directory):

  numerate = 1

  for filename in os.listdir(path_to_directory):
    f = os.path.join(path_to_directory, filename)
    if os.path.isfile(f):
      print(f"{numerate}. {f}")
      numerate = numerate + 1

In [ ]:
# Add the metadata
def add_metadata(path_to_file):
  reader = PdfReader(path_to_file)
  writer = PdfWriter()

  # Add all pages to the writer
  for page in reader.pages:
      writer.add_page(page)

  writer.add_metadata(
      {
          "/Author": "Martino",
          "/Producer": "Libre Writer",
          "/Subject": "kikk",
          "/Title": "jakis_tytul",
          # "/ModDate": modification_date  # Adding modification date
      }
  )

  with open(os.path.join(path_to_working_directory, path_to_file), "wb") as f:
    writer.write(f)

In [ ]:
def read_metadata(reader):
  meta = reader.metadata
  print(len(reader.pages))

  # All of the following could be None!
  print(f"author: {meta.author}")
  print(f"creator: {meta.creator}")
  print(f"producer: {meta.producer}")
  print(f"subject: {meta.subject}")
  print(f"title: {meta.title}")

In [ ]:
# reading form fields

def reading_form_fields(reader):
  # fields = reader.get_form_text_fields()
  # fields == {"key": "value", "key2": "value2"}
  # if reader.isEditable:
  fields = reader.get_fields()

  if bool(fields):
    # print(fields)
    for field_name, field_value in fields.items():
      print(f"nazwa pola: {field_name}, wartość: {field_value}")
  else:
    print("brak fields-ów")

  # for field_name in fields:
    # print(field_name)

In [ ]:
# extract text
def extract_text(reader):
  text = ""
  try:
      for page in reader.pages:
          text += page.extract_text()
  except Exception as exc:
      text = fallback_text_extraction("example.pdf")

  return text

In [ ]:
# sprawdzenie czy istnieje katalog roboczy, w którym chcemy pracować
def check_if_directory_and_path_exists(directory_name):
  # Ścieżka do katalogu bieżącego
  cwd = os.getcwd()
  print(f"katalog \"bieżący\": {cwd}")

  working_directory_for_creating = os.path.join(cwd, directory_name)
  print(f"katalog roboczy do utworzenia (jeśli nie istnieje): {working_directory_for_creating}")

  try:
    if not os.path.exists(working_directory_for_creating):
      print(f"katalog: {working_directory_for_creating} nie istnieje i zostanie utworzony")
      os.makedirs(working_directory_for_creating)
    else:
      print(f"katalog: {working_directory_for_creating} już istnieje!")
  except FileExistsError:
    print(f"katalog o nazwie {working_directory_for_creating} już istnieje!")
  except Exception as e:
    print(f"wystąpił błąd podczas tworzenie katalogu: {working_directory_for_creating}")

In [ ]:
def import_files(destination_directory):
  from google.colab import files

  uploaded = files.upload()
  # print(uploaded.keys())

  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

  for filename, file_content in uploaded.items():
    if os.path.exists(destination_directory) and os.path.isdir(destination_directory):
      with open(os.path.join(destination_directory, filename), "wb") as f:
        f.write(file_content)

      # remove files from /content
      file_to_remove_after_copying_to_working_directory = os.path.join(os.getcwd(), filename)
      if os.path.isfile(file_to_remove_after_copying_to_working_directory):
        os.remove(file_to_remove_after_copying_to_working_directory)
        if not os.path.exists(file_to_remove_after_copying_to_working_directory):
          print(f"plik: {os.path.basename(file_to_remove_after_copying_to_working_directory)} "
          f"usunięty z katalogu {os.path.dirname(file_to_remove_after_copying_to_working_directory)}")
        else:
          print(f"plik: {os.path.basename(file_to_remove_after_copying_to_working_directory)} "
          f"NIE usunięty z katalogu {os.path.dirname(file_to_remove_after_copying_to_working_directory)}")

In [ ]:
# usuwanie stron w pdf
def page_deleting(source_file, pages_nums_to_delete):
  print(source_file)

  file_name_without_ext, ext = os.path.splitext(source_file)
  new_file_name = ''.join([os.path.basename(file_name_without_ext), ext])

  destination_file = os.path.join(os.path.dirname(source_file), new_file_name)
  print(file_name_without_ext)
  print(ext)
  print(new_file_name)

  with open(source_file, "rb") as file:
    reader = PdfReader(file)
    writer = PdfWriter()

    for page_num in range(len(reader.pages)):
      if page_num + 1 not in pages_nums_to_delete:
        page_to_stay = reader.pages[page_num]
        writer.add_page(page_to_stay)

  with open(destination_file, "wb") as new_file:
    writer.write(new_file)

  with open(destination_file, "rb") as new_file:
    print(f"liczba stron nowego pliku: {len(PdfReader(new_file).pages)}")

In [ ]:
# check if file exists
def check_if_file_exists(path_to_file):
  try:
    with open(path_to_file, 'r') as file:
      print(f"Plik {path_to_file} istnieje i ma się całkiem nieźle ;-)")
      return True
  except FileNotFoundError:
    print('Sorry the file we\'re looking for doesn\' exist')
    return False
  except Exception as e:
    print(f"Wystąpił błąd: {e}")
    return False

In [ ]:
# upload files
def upload_files(folder_domyslny = "uploaded"):
  check_if_directory_and_path_exists(folder_domyslny)
  import_files(folder_domyslny)

In [ ]:
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from google.colab import files
from datetime import datetime

In [ ]:
# rozdziel nazwe pliku od rozszerzenia
def split_filename_and_extension(filename):
  nazwa_pliku, rozszerzenie_pliku = os.path.splitext(filename)

  return {'name_key': nazwa_pliku, 'ext_key': rozszerzenie_pliku}

In [ ]:
# wczytaj paragrafy z pliku docs - analiza zawartości

from docx import Document
doc = Document()

dokument_word_do_analizy = Document(os.path.join(os.getcwd(), "Zapytanie ofertowe RFX KZP-ŁA.2110.54.2023.docx"))
print(dokument_word_do_analizy)

for index, paragraf in enumerate(dokument_word_do_analizy.paragraphs):
  pass
  # print(f"\n\n--> Paragraf nr {index} \n", paragraf.text)

In [ ]:
doc = Document()

# modyfikacja stylu Normal
normal_style = doc.styles['Normal']
normal_style.font.name = 'Tahoma'
normal_style.font.size = Pt(9)
normal_style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.LEFT

# styl nagłówka GŁÓWNEGO
style = doc.styles.add_style('nagłówek_główny_styl', 1)
style.font.name = 'Tahoma'
style.font.size = Pt(9)
style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER

# zmiana marginesu storny
sekcja = doc.sections[0]
sekcja.left_margin = Inches(1)
sekcja.right_margin = Inches(1)
szerokosc_strony = sekcja.page_width - sekcja.left_margin - sekcja.right_margin

# dodanie logo firmy
logo_sciezka = '/content/uploaded/nagłowek_enea.PNG'

if not check_if_file_exists(logo_sciezka):
  upload_files(os.path.dirname(logo_sciezka))
  logo_added = doc.add_picture(logo_sciezka)
  wciecie_logo = szerokosc_strony - logo_added.width / 2
else:
  logo_added = doc.add_picture(logo_sciezka)
  wciecie_logo = szerokosc_strony - logo_added.width / 2

# dane wejściowe
nazwa_sprawy = "KZP.ŁA.2110.23.2024 Dostawa łożysk"
data_dokumentu = datetime.now().strftime('%d.%m.%Y')
# data_dokumentu = datetime.now().strftime('%d.%m.%Y %H:%M:%S')

# dodanie znaku sprawy i daty
p = doc.add_paragraph(f'Nasz znak: ')
run_znak_sprawy = p.add_run(f'{nazwa_sprawy}')
run_znak_sprawy.bold = True
p.style = 'Normal'

# dodanie daty dokumentu
data = doc.add_paragraph(f"Świerże Górne, dn. ")
data_run = data.add_run(f'{data_dokumentu}r.')
data_run.bold = True
data.alignment = WD_ALIGN_PARAGRAPH.RIGHT

# dodanie nagłówka
nagłówek_text = doc.add_heading('ZAPYTANIE OFERTOWE (RFX)', 0)
nagłówek_text.style = 'nagłówek_główny_styl'

pkt_1 = "WPROWADZENIE:\nNiniejsze Zapytanie jest podstawą do złożenia ofert."
pkt_2 = "TRYB UDZIELENIA ZAMÓWIENIA:\nPostępowanie w trybie Zapytania Ofertowego (RFX) prowadzone jest w oparciu o wewnętrzny Regulamin Udzielania Zamówień i Zawierania Umów (RUZiZU) Zamawiającego, dla którego nie mają zastosowania przepisy ustawy z dnia 11.09.2019 r. Prawo zamówień publicznych."
podpunkt_3_1 = f"Wykonanie usługi pn. \"{nazwa_sprawy}\"."
podpunkt_3_2 = "Zakres dostawy wg: Załącznika Nr 1 pn. \"Zakres Przedmiotu zamówienia\"."
pkt_3 = {"PRZEDMIOT ZAMÓWIENIA": [podpunkt_3_1, podpunkt_3_2]}
pkt_4 = "OFERTY CZĘŚCIOWE:\nZamawiający dopuszcza składania ofert częściowych."
pkt_5 = "OFERTY WARIANTOWE:\nZamawiający nie dopuszcza składania ofert wariantowych."
podpunkt_6_1 = "rozpoczęcie wykonywania Przedmiotu umowy  – od dnia podpisania Zamówienia,"
podpunkt_6_2 = f"zakończenie wykonywania Przedmiotu umowy – zgodnie z załącznikiem „1. Załącznik {nazwa_sprawy} — zakres przedmiotu zamówienia”; lub 1 tydzień od daty otrzymania zamówienia, termin ten uważany będzie za dochowany przez Wykonawcę w przypadku, gdy przed jego upływem nastąpi protokolarny, bezusterkowy odbiór końcowy prac będących Przedmiotem umowy,"
pkt_6 = {"TERMINY REALIZACJI: ":[podpunkt_6_1, podpunkt_6_2]}
podpunkt_7_1 = "Realizacja przedmiotu zamówienia polega na:  Dostawie dla Enea Wytwarzanie sp. z o.o. fabrycznie nowych, nieużywanych, nieregenerowanych wyrobów będących przedmiotem niniejszego zapytania ofertowego wykonanych zgodnie z opisem przedmiotu zamówienia."
pkt_7 = {"SZCZEGÓŁOWE WARUNKI TECHNICZNE/TECHNOLOGICZNE REALIZACJI PRZEDMIOTU ZAMÓWIENIA":[podpunkt_7_1]}

lista_punktów = [pkt_1, pkt_2, pkt_3, pkt_4, pkt_5, pkt_6, pkt_7]

for punkt in lista_punktów:
  if isinstance(punkt, dict):
    for main_item, sub_items in punkt.items():
      paragraf = doc.add_paragraph(main_item, style = 'ListNumber')
      for sub_item in sub_items:
        sub_paragraf = doc.add_paragraph(sub_item, style = 'ListBullet')
        sub_paragraf.paragraph_format.left_indent = 360
  else:
    paragraf = doc.add_paragraph(punkt, style = 'ListNumber')

doc.add_page_break()

# złożenie nowej nazwy pliku wraz z nazwą sprawy
podzielona_nazwa_pliku_slownik = split_filename_and_extension("Zapytanie_ofertowe_RFX.docx")
nazwa_trzon = podzielona_nazwa_pliku_slownik['name_key']
rozszerzenie = podzielona_nazwa_pliku_slownik['ext_key']

skladowe_nazwy_pliku = [nazwa_trzon, "_", nazwa_sprawy, rozszerzenie]
nowa_nazwa_pliku = "".join(skladowe_nazwy_pliku)
print(nowa_nazwa_pliku)

doc.save(nowa_nazwa_pliku)
files.download(nowa_nazwa_pliku)

Plik /content/uploaded/nagłowek_enea.PNG istnieje i ma się całkiem nieźle ;-)
Zapytanie_ofertowe_RFX_KZP.ŁA.2110.23.2024 Dostawa łożysk.docx


/usr/local/lib/python3.10/dist-packages/docx/styles/styles.py:130: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def actual_datetime():
  teraz = datetime.now(tz=tz.gettz('Europe/Warsaw'))
  formatted_date = teraz.strftime("%d_%m_%Y")
  formatted_time = teraz.strftime("%H_%M_%S")

  # print(formatted_date)
  # print(formatted_time)

  return {'data': formatted_date, 'czas': formatted_time}


In [ ]:
from openpyxl import Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from dateutil import tz
from datetime import datetime  # Import the datetime module

# utworzenie katalogu do pracy z biblioteka openpyxl

try:
  dir_name_for_openpyxl = "openpyxl"
  check_if_directory_and_path_exists(dir_name_for_openpyxl)

  wb = Workbook()
  ws = wb.active

  data = [
      ['Apples', 10000, 5000, 8000, 6000],
      ['Pears',   2000, 3000, 4000, 5000],
      ['Bananas', 6000, 6000, 6500, 6000],
      ['Oranges',  500,  300,  200,  700],
  ]

  # add column headings. NB. these must be strings
  ws.append(["Fruit", "2011", "2012", "2013", "2014"])
  for row in data:
      ws.append(row)

  tab = Table(displayName="Table1", ref="A1:E5")

  # Add a default style with striped rows and banded columns
  style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                        showLastColumn=False, showRowStripes=True, showColumnStripes=True)
  tab.tableStyleInfo = style

  '''
  Table must be added using ws.add_table() method to avoid duplicate names.
  Using this method ensures table name is unque through out defined names and all other table name.
  '''
  ws.add_table(tab)

  # Save the file with actual date and time
  data_czas = actual_datetime()
  lista_daty = ["_", data_czas['czas'], "___", data_czas['data']]
  skrot_do_daty_i_godziny = "".join(lista_daty)

  podzielona_nazwa_pliku = split_filename_and_extension("sample.xlsx")
  nowa_nazwa_pliku_skladowe = [podzielona_nazwa_pliku.get('name_key'),
                               skrot_do_daty_i_godziny,
                               podzielona_nazwa_pliku.get('ext_key')]

  nowa_nazwa_pliku = "".join(nowa_nazwa_pliku_skladowe)

  wb.save(nowa_nazwa_pliku)

except Exception as e:
  print(f"wyje.ało błęda: {e}")

katalog "bieżący": /content
katalog roboczy do utworzenia (jeśli nie istnieje): /content/openpyxl
katalog: /content/openpyxl już istnieje!
